In [1]:
import numpy as np
import h5py
import scipy.io
import random
from keras.preprocessing import sequence
from keras.optimizers import RMSprop
from keras.models import Sequential
from keras.models import load_model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from keras.regularizers import l2, activity_l1
from keras.constraints import maxnorm
from keras.layers.recurrent import LSTM, GRU
from sklearn.metrics import average_precision_score, roc_auc_score
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

Using TensorFlow backend.


In [2]:
def seq_matrix(seq_list,label):
    tensor = np.zeros((len(seq_list),203,8))
    for i in range(len(seq_list)):
        seq = seq_list[i]
        j = 0
        for s in seq:
            if s == 'A' and (j<100 or j>102):
                tensor[i][j] = [1,0,0,0,0,0,0,0]
            if s == 'T' and (j<100 or j>102):
                tensor[i][j] = [0,1,0,0,0,0,0,0]
            if s == 'C' and (j<100 or j>102):
                tensor[i][j] = [0,0,1,0,0,0,0,0]
            if s == 'G' and (j<100 or j>102):
                tensor[i][j] = [0,0,0,1,0,0,0,0]
            if s == '$':
                tensor[i][j] = [0,0,0,0,0,0,0,0]
            if s == 'A' and (j>=100 and j<=102):
                tensor[i][j] = [0,0,0,0,1,0,0,0]
            if s == 'T' and (j>=100 and j<=102):
                tensor[i][j] = [0,0,0,0,0,1,0,0]
            if s == 'C' and (j>=100 and j<=102):
                tensor[i][j] = [0,0,0,0,0,0,1,0]
            if s == 'G' and (j>=100 and j<=102):
                tensor[i][j] = [0,0,0,0,0,0,0,1]
            j += 1
    if label == 1:
        y = np.ones((len(seq_list),1))
    else:
        y = np.zeros((len(seq_list),1))
    return tensor, y

In [3]:
codon_tis_prior = np.load('TITER/codes/dict_piror_front_Gaotrain.npy',allow_pickle=True)
codon_tis_prior = codon_tis_prior.item()

codon_list = []
for c in codon_tis_prior.keys():
    if codon_tis_prior[c]!='never' and codon_tis_prior[c] >= 1:
        codon_list.append(c)

## Average Accuracy on Original Test Dataset after Data Balancing

In [4]:
positives= [i.decode("utf-8") for i in np.load('TITER/data/data/pos_seq_test.npy')]
negatives = [i.decode("utf-8") for i in np.load('TITER/data/data/neg_seq_test_all_upstream.npy')]

accuracies_p=[]
accuracies_n=[]

runs=100

for run in range(runs):
    
    pos_seq_test = np.array(positives)
    neg_seq_test = np.array(negatives)
    pos_codon = []
    neg_codon = []
    
    
    neq_seq_test_dec = []
    for i in np.arange(len(neg_seq_test)):
        neq_seq_test_dec.append(neg_seq_test[i])
    
    pos_seq_test_dec = []
    for i in np.arange(len(pos_seq_test)):
        pos_seq_test_dec.append(pos_seq_test[i])

      
    pos_seq_test1 = []
    neg_seq_test1 = []
    for s in pos_seq_test_dec:
        if s[100:103] in codon_list:
            pos_seq_test1.append(s)
    for s in neq_seq_test_dec:
        if s[100:103] in codon_list:
            neg_seq_test1.append(s)            


    #Here, we randomly sample negatives to balanced the number of positives and negatives used.

    neg_seq_test1 = random.sample(neg_seq_test1,len(pos_seq_test1))
    
    
            
    for s in pos_seq_test1:
        if s[100:103] in codon_list:
            pos_codon.append(codon_tis_prior[s[100:103]])
    for s in neg_seq_test1:
        if s[100:103] in codon_list:
            neg_codon.append(codon_tis_prior[s[100:103]])
    
    pos_codon = np.array(pos_codon)
    neg_codon = np.array(neg_codon)
    codon = np.concatenate((pos_codon,neg_codon)).reshape((len(pos_codon)+len(neg_codon),1))
        
    pos_test_X, pos_test_y = seq_matrix(seq_list=pos_seq_test1, label=1)
    neg_test_X, neg_test_y = seq_matrix(seq_list=neg_seq_test1, label=0)
    X_test = np.concatenate((pos_test_X,neg_test_X), axis=0)
    y_test = np.concatenate((pos_test_y,neg_test_y), axis=0)
    
    model = Sequential()
    
    model.add(Convolution1D(nb_filter=128,
                            filter_length=3,
                            input_dim=8,
                            input_length=203,
                            border_mode='valid',
                            W_constraint = maxnorm(3),
                            activation='relu',
                            subsample_length=1))
    model.add(MaxPooling1D(pool_length=3))
    model.add(Dropout(p=0.21370950078747658))
    model.add(LSTM(output_dim=256,
                   return_sequences=True))
    model.add(Dropout(p=0.7238091317104384))
    model.add(Flatten())
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    
    model.compile(loss='binary_crossentropy',
                  optimizer='nadam',
                  metrics=['accuracy'])
    
    y_test_pred_n = np.zeros((len(y_test),1))
    y_test_pred_p = np.zeros((len(y_test),1))
    
    for i in range(32):
        model.load_weights('TITER/model/bestmodel_'+str(i)+'.hdf5')
        y_test_pred = model.predict(X_test,verbose=0)
        y_test_pred_n += y_test_pred
        y_test_pred_p += y_test_pred*codon
    
    y_test_pred_n = y_test_pred_n/32
    y_test_pred_p = y_test_pred_p/32
    
    
    predictions_n = []
    for n in y_test_pred_n:
        if n>0.5:
            predictions_n+=[1]
        else:
            predictions_n+=[0]        
    
    predictions_p = []
    for p in y_test_pred_p:
        if p>0.5:
            predictions_p+=[1]
        else:
            predictions_p+=[0]
    
    accuracies_n+=[sum(np.array(predictions_n)==y_test.reshape(1,len(y_test))[0])/len(y_test)]
            
    accuracies_p+=[sum(np.array(predictions_p)==y_test.reshape(1,len(y_test))[0])/len(y_test)]

In [5]:
print('Average accuracy without prior:',np.mean(accuracies_n))
        
print('Average accuracy with prior:',np.mean(accuracies_p))

Average accuracy without prior: 0.6368187744458931
Average accuracy with prior: 0.6693872229465451


## Accuracy on ATG RFC Test Dataset

In [6]:
positives_atg = np.load('TITER Test Data/test_aug_pos.npy',allow_pickle=True)
negatives_atg = np.load('TITER Test Data/test_aug_neg.npy',allow_pickle=True)


pos_seq_test = positives_atg
neg_seq_test = negatives_atg
pos_codon = []
neg_codon = []



neq_seq_test_dec = []
for i in np.arange(len(neg_seq_test)):
    neq_seq_test_dec.append(neg_seq_test[i])

pos_seq_test_dec = []
for i in np.arange(len(pos_seq_test)):
    pos_seq_test_dec.append(pos_seq_test[i])

for s in pos_seq_test_dec:
    if s[100:103] in codon_list:
        pos_codon.append(codon_tis_prior[s[100:103]])
for s in neq_seq_test_dec:
    if s[100:103] in codon_list:
        neg_codon.append(codon_tis_prior[s[100:103]])

pos_codon = np.array(pos_codon)
neg_codon = np.array(neg_codon)
codon = np.concatenate((pos_codon,neg_codon)).reshape((len(pos_codon)+len(neg_codon),1))
  
pos_seq_test1 = []
neg_seq_test1 = []
for s in pos_seq_test_dec:
    if s[100:103] in codon_list:
        pos_seq_test1.append(s)
for s in neq_seq_test_dec:
    if s[100:103] in codon_list:
        neg_seq_test1.append(s)

pos_test_X, pos_test_y = seq_matrix(seq_list=pos_seq_test1, label=1)
neg_test_X, neg_test_y = seq_matrix(seq_list=neg_seq_test1, label=0)
X_test = np.concatenate((pos_test_X,neg_test_X), axis=0)
y_test = np.concatenate((pos_test_y,neg_test_y), axis=0)

model = Sequential()

model.add(Convolution1D(nb_filter=128,
                        filter_length=3,
                        input_dim=8,
                        input_length=203,
                        border_mode='valid',
                        W_constraint = maxnorm(3),
                        activation='relu',
                        subsample_length=1))
model.add(MaxPooling1D(pool_length=3))
model.add(Dropout(p=0.21370950078747658))
model.add(LSTM(output_dim=256,
               return_sequences=True))
model.add(Dropout(p=0.7238091317104384))
model.add(Flatten())
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='nadam',
              metrics=['accuracy'])

y_test_pred_n = np.zeros((len(y_test),1))
y_test_pred_p = np.zeros((len(y_test),1))

for i in range(32):
    model.load_weights('TITER/model/bestmodel_'+str(i)+'.hdf5')
    y_test_pred = model.predict(X_test,verbose=0)
    y_test_pred_n += y_test_pred
    y_test_pred_p += y_test_pred*codon

y_test_pred_n = y_test_pred_n/32
y_test_pred_p = y_test_pred_p/32

print ('Perf without prior, AUC: '+str(roc_auc_score(y_test, y_test_pred_n)))
print ('Perf without prior, AUPR: '+str(average_precision_score(y_test, y_test_pred_n)))
print ('Perf with prior, AUC: '+str(roc_auc_score(y_test, y_test_pred_p)))
print ('Perf with prior, AUPR: '+str(average_precision_score(y_test, y_test_pred_p)))

Perf without prior, AUC: 0.6990265008112493
Perf without prior, AUPR: 0.6701060106592327
Perf with prior, AUC: 0.6990265008112493
Perf with prior, AUPR: 0.6701060106592327


In [7]:
predictions_n = []
for i in y_test_pred_n:
    if i>0.5:
        predictions_n+=[1]
    else:
        predictions_n+=[0]        

predictions_p = []
for i in y_test_pred_p:
    if i>0.5:
        predictions_p+=[1]
    else:
        predictions_p+=[0]

print('Accuracy without prior:',sum(np.array(predictions_n)==y_test.reshape(1,len(y_test))[0])/len(y_test))
        
print('Accuracy with prior:',sum(np.array(predictions_p)==y_test.reshape(1,len(y_test))[0])/len(y_test))

Accuracy without prior: 0.622093023255814
Accuracy with prior: 0.563953488372093


## Accuracy on Near-Cognate RFC Test Dataset

In [8]:
positives_nc = np.load('TITER Test Data/test_near-cognate_pos.npy',allow_pickle=True)
negatives_nc = np.load('TITER Test Data/test_near-cognate_neg.npy',allow_pickle=True)

pos_seq_test = np.array(positives_nc)
neg_seq_test = np.array(negatives_nc)
pos_codon = []
neg_codon = []


neq_seq_test_dec = []
for i in np.arange(len(neg_seq_test)):
    neq_seq_test_dec.append(neg_seq_test[i])

pos_seq_test_dec = []
for i in np.arange(len(pos_seq_test)):
    pos_seq_test_dec.append(pos_seq_test[i])

for s in pos_seq_test_dec:
    if s[100:103] in codon_list:
        pos_codon.append(codon_tis_prior[s[100:103]])
for s in neq_seq_test_dec:
    if s[100:103] in codon_list:
        neg_codon.append(codon_tis_prior[s[100:103]])

pos_codon = np.array(pos_codon)
neg_codon = np.array(neg_codon)
codon = np.concatenate((pos_codon,neg_codon)).reshape((len(pos_codon)+len(neg_codon),1))
  
pos_seq_test1 = []
neg_seq_test1 = []
for s in pos_seq_test_dec:
    if s[100:103] in codon_list:
        pos_seq_test1.append(s)
for s in neq_seq_test_dec:
    if s[100:103] in codon_list:
        neg_seq_test1.append(s)

pos_test_X, pos_test_y = seq_matrix(seq_list=pos_seq_test1, label=1)
neg_test_X, neg_test_y = seq_matrix(seq_list=neg_seq_test1, label=0)
X_test = np.concatenate((pos_test_X,neg_test_X), axis=0)
y_test = np.concatenate((pos_test_y,neg_test_y), axis=0)

model = Sequential()

model.add(Convolution1D(nb_filter=128,
                        filter_length=3,
                        input_dim=8,
                        input_length=203,
                        border_mode='valid',
                        W_constraint = maxnorm(3),
                        activation='relu',
                        subsample_length=1))
model.add(MaxPooling1D(pool_length=3))
model.add(Dropout(p=0.21370950078747658))
model.add(LSTM(output_dim=256,
               return_sequences=True))
model.add(Dropout(p=0.7238091317104384))
model.add(Flatten())
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='nadam',
              metrics=['accuracy'])

y_test_pred_n = np.zeros((len(y_test),1))
y_test_pred_p = np.zeros((len(y_test),1))

for i in range(32):
    model.load_weights('TITER/model/bestmodel_'+str(i)+'.hdf5')
    y_test_pred = model.predict(X_test,verbose=0)
    y_test_pred_n += y_test_pred
    y_test_pred_p += y_test_pred*codon

y_test_pred_n = y_test_pred_n/32
y_test_pred_p = y_test_pred_p/32

print ('Perf without prior, AUC: '+str(roc_auc_score(y_test, y_test_pred_n)))
print ('Perf without prior, AUPR: '+str(average_precision_score(y_test, y_test_pred_n)))
print ('Perf with prior, AUC: '+str(roc_auc_score(y_test, y_test_pred_p)))
print ('Perf with prior, AUPR: '+str(average_precision_score(y_test, y_test_pred_p)))

Perf without prior, AUC: 0.6002777777777778
Perf without prior, AUPR: 0.6639714816691354
Perf with prior, AUC: 0.6025
Perf with prior, AUPR: 0.6138585640701851


In [9]:
predictions_n = []
for i in y_test_pred_n:
    if i>0.5:
        predictions_n+=[1]
    else:
        predictions_n+=[0]        

predictions_p = []
for i in y_test_pred_p:
    if i>0.5:
        predictions_p+=[1]
    else:
        predictions_p+=[0]

print('Accuracy without prior:',sum(np.array(predictions_n)==y_test.reshape(1,len(y_test))[0])/len(y_test))
        
print('Accuracy with prior:',sum(np.array(predictions_p)==y_test.reshape(1,len(y_test))[0])/len(y_test))

Accuracy without prior: 0.575
Accuracy with prior: 0.5833333333333334
